In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from deap import tools, algorithms

from preprocessing import preprocess_data
from ga_setup import setup_toolbox, setup_stats

In [2]:
df = pd.read_csv("./data/wig_d.csv", sep=";")
df = preprocess_data(df)

In [3]:
# for col in df.columns:
#     plt.plot(df[col])
#     plt.title(col)
#     plt.show()

In [4]:
stock_data = df.iloc[:, :6]
ta_features = df.iloc[:, 6:]

In [5]:
toolbox = setup_toolbox(stock_data, ta_features)
stats = setup_stats()
hof = tools.HallOfFame(1)
population = toolbox.population(n=50)
algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=50, halloffame=hof, stats=stats, verbose=True)

gen	nevals	avg            	std           	min           	max            
0  	50    	[1149.33913979]	[187.74376284]	[829.65130005]	[1721.83742486]
1  	24    	[1278.35094158]	[213.86832356]	[882.57342788]	[1721.83742486]
2  	30    	[1434.9375807] 	[225.1886986] 	[902.17073777]	[1721.83742486]
3  	26    	[1565.47415711]	[214.41539526]	[1063.03995195]	[1857.57314839]
4  	28    	[1696.68170492]	[135.26365199]	[1124.33174769]	[1858.2740725] 
5  	30    	[1761.31459372]	[145.64278054]	[1235.56993577]	[2227.77225735]
6  	32    	[1798.07319673]	[80.08159949] 	[1413.51643874]	[1881.45965693]
7  	34    	[1755.88321582]	[196.34163459]	[957.3975312]  	[1933.09850763]
8  	39    	[1826.26339002]	[108.19071065]	[1473.83397959]	[2010.87852671]
9  	31    	[1865.51636874]	[93.32066246] 	[1562.89368967]	[2037.67000018]
10 	29    	[1888.30543219]	[84.30360239] 	[1664.57704316]	[2037.67000018]
11 	22    	[1939.0695505] 	[57.41687939] 	[1710.95435764]	[2037.67000018]
12 	34    	[1924.35573921]	[91.08781038] 	

([[0.17475252042261147,
   -0.32749103499400545,
   -0.7097706652703744,
   -0.552635475299563,
   -0.2815049858351393,
   -0.01439281385552671,
   -0.5866029643885182,
   0.3074615678720847,
   0.4796720728921855,
   0.19252418159188478,
   0.004355756538533262,
   -0.36877137006577443,
   -0.22307818269173585,
   0.09133945012494893,
   -0.2844954959814219,
   -0.628130194646815,
   0.648783699958523,
   -0.7869238873306683,
   0.20779984428928144,
   0.9155962349066848,
   -0.2391637577015866,
   0.317202955093417,
   -0.28663631641314136,
   -0.8851204372262955,
   -0.9360769039780468,
   0.6958853549352182,
   0.5426131213055945,
   -0.4528430679749116,
   -0.9621686251928359,
   -0.41406525280569184,
   0.15683223922435974,
   0.3655496259457356,
   0.4175956191347894,
   0.4423970016985226,
   -0.560609255242368,
   0.03766199257828484,
   -0.16268165985632477,
   0.451144107415294,
   0.09123091694656082,
   -0.33522894993932284,
   -0.19892638641203986,
   0.8184423085949597,


In [7]:
hof[0]

[0.17475252042261147,
 -0.32749103499400545,
 -0.7147037364072046,
 -0.552635475299563,
 -0.2815049858351393,
 -0.01439281385552671,
 -0.5866029643885182,
 0.3074615678720847,
 0.4796720728921855,
 0.19252418159188478,
 0.004355756538533262,
 -0.36877137006577443,
 -0.22307818269173585,
 0.09133945012494893,
 -0.2844954959814219,
 -0.628130194646815,
 0.648783699958523,
 -0.7869238873306683,
 0.20779984428928144,
 0.9155962349066848,
 -0.2391637577015866,
 0.317202955093417,
 -0.28663631641314136,
 -0.8851204372262955,
 -0.9360769039780468,
 0.6958853549352182,
 0.5426131213055945,
 -0.4528430679749116,
 -0.9621686251928359,
 -0.41406525280569184,
 0.15683223922435974,
 0.3655496259457356,
 0.4175956191347894,
 0.4423970016985226,
 -0.560609255242368,
 0.03766199257828484,
 -0.16268165985632477,
 0.451144107415294,
 0.09123091694656082,
 -0.33522894993932284,
 -0.19892638641203986,
 0.8184423085949597,
 -0.29385056940813703,
 -0.4029429185727018,
 0.29507674460090483,
 -0.5129855194667